<a href="https://colab.research.google.com/github/Sripathm2/UCLA_CS_245_Project5/blob/GNN/GNN/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install spektral

In [163]:
import numpy as np
from scipy import sparse
import tensorflow as tf
import spektral
from spektral.layers.ops import sp_matrix_to_sp_tensor
from spektral.datasets.mnist import MNIST

data = MNIST()


class Net(tf.keras.Model):
    def __init__(self, window=6, dropout=.5, **kwargs):
        """
        Window: int. Window of days
        #LSTM hidden states: 64
        Training: 500 epocs, batchsize 8, Adam optimizer, LR 10-3
        """
        super().__init__(**kwargs)
        self._nets = self.build_MPNN_unit(dropout)
        self.permute = tf.keras.layers.Permute((2,1))
        self.flatten = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())
        self.LSTM1 = tf.keras.layers.LSTM(52, return_sequences=True)
        self.LSTM2 = tf.keras.layers.LSTM(52, return_sequences=False,
                                          return_state=True)

    def build_MPNN_unit(self, dropout):
        L1 = []
        L1.append(
            spektral.layers.MessagePassing(aggregate='sum',
                                           activation='relu')
            )
        L1.append(
            tf.keras.layers.BatchNormalization()
            )
        L1.append(
            tf.keras.layers.Dropout(dropout)
            )
        L2 = []
        L2.append(
            spektral.layers.MessagePassing(aggregate='sum',
                                           activation='relu')
            )
        L2.append(
            tf.keras.layers.BatchNormalization()
            )
        L2.append(
            tf.keras.layers.Dropout(dropout)
            )
        return (L1,L2)


    def run_MPNN_unit(self, Adj, X):
        L1, L2 = self._nets
        y = None
        for i in range(0,len(L1)):
            if i == 0: # MessagePassing layer
                y = L1[i].propagate(X, Adj)
                continue
            # print(i,L1[i])#, y)
            y = L1[i](y)
        H1 = y
        for i in range(0, len(L2)):
            if i == 0: # MessagePassing Layer
                y = L2[i].propagate(y, Adj)
                continue
            y = L2[i](y)
        H2 = y
        return tf.concat((H1,H2), axis=1)
    
    def call(self, inputs):
        Adj, X = inputs
        H_list = []
        for i in range(Adj.shape[0]):
          a = sp_matrix_to_sp_tensor(Adj[i])
          H = self.run_MPNN_unit(a, X[i])
          H_list.append(H)
        H_out = tf.expand_dims(H_list, axis=0)
        #LSTM_input = self.permute(H_out)[0]
        LSTM_input = self.flatten(H_out)
        x = self.LSTM1(inputs=LSTM_input)
        x, final_memory_state, final_carry_state = self.LSTM2(inputs=x)
        x = tf.transpose(x)
        print('Output shape of LSTM: ',x.shape)
        print('Output of LSTM: ', x)
        #x = X+x
        #Lin?
        #x = tf.keras.activations.relu(x)
        return x

# Create random Adj Matrices
A_list = []
for i in range(286):
  temp = np.zeros([52,52])
  for j in range(np.random.randint(1,25)):
    r = np.random.randint(0,52)
    s = np.random.randint(0,52)
    n = np.random.randn()
    temp[r,s] = n
    temp[s,r] = n
  A_list.append(temp)
Adj = tf.expand_dims(A_list, axis=0)[0]

# Create random node feature matrices
X = [np.random.rand(52, 5) for i in range(0,286)]
X = tf.expand_dims(X, axis=0)[0]

input = [Adj, X]
# Create random labels
y = np.random.rand(52, 1)

model = Net(window=6)
model.compile(optimizer=tf.keras.optimizers.Adam(),metrics='mse', loss=tf.keras.metrics.MeanSquaredError())
model.fit(x=input, y=y, epochs=5)
print('end')

ValueError: ignored